In [1]:
%load_ext sql
%config SqlMagic.autopandas=False

/home/juliano/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


$Airport(\underline{apID}, name, city, country)$

$AirportDistance(\underline{apID1, apID2}, distance)$

$Airline(\underline{cID}, callSign, headquarters, mainHub, website)$

$Aircraft(\underline{acID}, type, avgSpeed, maxSpeed, range, serialNumber, firstFlightDate, manufacturer)$

$Employee(\underline{eID}, name, airline, type, rank, primary\_language, secondary\_language, startDate)$

$Flight(\underline{cID, flightNumber}, origin, destination, departureTime, arrivalTime)$

$FlightInstance(\underline{fID}, cID, flightNumber, date, acID, actualDepartureTime, actualArrivalTime)$

$CrewAssignment(\underline{fID, eID}, role)$

$PilotExpertise(\underline{eID}, aircraftType)$

$\Pi, \sigma, \bowtie, \bowtie_{condition}, \times, \cap, \cup, -, \rho, (such as \leq and \neq)$
and boolean operators $(\vee, \wedge and \neg).$

### 1. List for each airline the employees who have expertise in at least two different types of aircraft. Use the two-character airline code.


$EmpExpertise(eID, airline, aircraftType) := Employee \bowtie PilotExpertise$

$Experts := \sigma_{E1.eID = E2.eID \wedge E1.aircraftType \neq E2.aircraftType}[(\rho_{E1} EmpExpertise) \times (\rho_{E2} EmpExpertise)]$

$Answer := \Pi_{eID, airline} Experts$

### 2. Find all the airports (airport code and name) that one can reach on the same day if one departs from Pearson airport (apID ’YYZ’) and is willing to take at most 3 flights. Assume that one needs at least 120 minutes between consecutive flights. 

**OBS**: If One.at + 120 <= FlInfo.dt, it means we can make both flights in the same day since every flight starts and finishes on the same day. Also, can I assume that an airport only travels to different places? maybe not


$FlInfo(cID, fnum, o, d, dt, at) := \Pi_{cID, flightNumber, origin, destination, departureTime, arrivalTime}Flight $

$One := \sigma_{o='YYZ'}FlInfo$

$Two := (One \bowtie_{((One.cID \neq FlInfo.cID)\; \vee \;(One.fnum \neq FlInfo.fnum))\; \wedge \;(One.d = FlInfo.o)\; \wedge \; (FlInfo.d \neq 'YYZ')} FlightInfo)$

$Two(cID, fnum, o, d, dt, at) := \Pi_{FlInfo.cID, FlInfo.fnum, FlInfo.o, FlInfo.d, FlInfo.dt, FlInfo.at}\sigma_{One.at + 120 \leq FlInfo.dt} Two$

$Three := (Two \bowtie_{((Two.cID \neq FlInfo.cID)\; \vee \;(Two.fnum \neq FlInfo.fnum))\wedge \;(Two.d = FlInfo.o)} FlightInfo)$

$Three(d) := \Pi_{FlInfo.d}\sigma_{Three.at + 120 \leq FlInfo.dt} Three$

$OneFlight(apID, name) := \Pi_{apID, name} (Airport \bowtie_{apID = d} (\Pi_{d} One))$

$TwoFlight(apID, name) := \Pi_{apID, name} (Airport \bowtie_{apID = d} (\Pi_{d} Two))$

$ThreeFlight(apID, name) := \Pi_{apID, name} (Airport \bowtie_{apID = d} (\Pi_{d} Three))$

$Answer(apID, name) := OneFlight\; \cup \; TwoFlight \; \cup \; ThreeFlight$


### 3. Find all flight instances on which no crew member speaks French (as either a first or second language). Return the airline, flight number, and the date of the flight instance. (Flight instances with no crew assigned satisfy the condition and should be returned as well.)

$AtLeastOneFrench(fID) :=  \Pi_{fID}(CrewAssignment \bowtie (\Pi_{eID}\sigma_{primary\_language='French' \; \vee \; secondary\_language='French'}Employee))$

$AllFlightsInstances (fID) := \Pi_{fID} FlightInstance$

$NoFrenchFlights(fID) := AllFlightsInstances - AtLeastOneFrench$

$Answer(cID, flightNumber, date) := \Pi_{cID, flightNumber, date} (FlightInstance \bowtie NoFrenchFlights)$

### 4. List all airports with more than 3 departing flights a day.

IDEA: Get all possible 4 tuples of flights, such that every flight is different from each other. Then, get the airport in which all agree, i.e. if all flights have the same 'origin', it means this airport has MORE than 3 departing flights (that is, at LEAST 4 flights departed a day, since a flight starts and ends at the same day).
The whole **Temp** relations are just to make sure every flight is paired only with different flights

$FlightSimplified(cid, flightNumber, origin) := \Pi_{cid, flightNumber, origin} Flight$

$QuadriCross := (\rho_{F1} FlightSimplified) \times (\rho_{F2} FlightSimplified) \times (\rho_{F3} FlightSimplified) \times (\rho_{F4} FlightSimplified)$

$Temp1 := \sigma_{\neg(F1.cid = F2.cid \wedge F1.flightNumber = F2.flightNumber) \wedge \neg(F1.cid = F3.cid \wedge F1.flightNumber = F3.flightNumber)} QuadriCross$

$Temp1 := \sigma_{\neg(F2.cid = F3.cid \wedge F2.flightNumber = F3.flightNumber)} Temp1$

$Temp2 := \sigma_{\neg(F1.cid = F4.cid \wedge F1.flightNumber = F4.flightNumber) \wedge  \neg(F2.cid = F4.cid \wedge F2.flightNumber = F4.flightNumber)} Temp1$

$AllDiffFlights := \sigma_{\neg(F3.cid = F4.cid \wedge F3.flightNumber = F4.flightNumber)} Temp2$


$Answer(airport) := \Pi_{F1.origin} \sigma_{F1.origin = F2.origin \wedge F2.origin = F3.origin  \wedge F3.origin = F4.origin} AllDiffFlights$

### 5. Find all pairs of employees who only work together. For example, the query should return a pair of employees 1 and 2 if employee 1 only works on flight instances on which employee 2 also works and vice versa. Return only the employee identifiers. (The employees returned must work on at least one flight instance.)